In [1]:
!pip install -r ../requirements.txt

In [2]:
import pandas as pd
from data.parliamentApiHelpers import ParliamentAPI
from data.generateJson import get_links
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np

In [11]:
frac_party_to_short = {
    "Eesti Keskerakonna fraktsioon": "KESK",
    "Eesti Konservatiivse Rahvaerakonna fraktsioon": "EKRE",
    "Eesti Reformierakonna fraktsioon": "REF",
    "Fraktsiooni mittekuuluvad Riigikogu liikmed": "MITTEKUULUVAD",
    "Isamaa fraktsioon": "I",
    "Sotsiaaldemokraatliku Erakonna fraktsioon": "SDE"
}
resp_party_to_short = {
    "Keskerakond": "KESK",
    "EKRE": "EKRE",
    "Reformierakond": "REF",
    "Isamaa": "I",
    "SDE": "SDE",
    "Parempoolsed": "PAR",
    "Rohelised": "ROH",
    "Eesti 200": "E200",
    "Eestimaa Ühendatud Vasakpartei": "EÜVP",
    "Üksikkandidaat": "ÜKS"
}

In [30]:
df_responses = pd.read_csv("../haaleandja/data_kandidaatide_vastused.csv")
df_responses["party_short"] = df_responses.party.map(resp_party_to_short)
df_responses["id"] = df_responses["name"] + "_" + df_responses["party_short"]

In [78]:
datasets = ["50", "51", "52", "XIV"]
dataset_to_score = {d:0 for d in datasets}
for dataset in datasets:
    data_folder = f"../coalition_{dataset}"
    df_votes = pd.read_parquet(f"{data_folder}/votes.parquet")
    df_voters = pd.read_parquet(f"{data_folder}/voters.parquet")
    df_voters["party_short"] = df_voters["faction"].map(frac_party_to_short)
    df_voters["id"] = df_voters["name"] + "_" + df_voters["party_short"]
    present_in_both = set(df_responses.id.unique()).intersection(df_voters.id.unique())
    df_votes_subselected = df_votes.rename(columns=df_voters["id"].to_dict())[list(present_in_both)]
    df_responses_subselected = df_responses[df_responses.id.isin(present_in_both)].set_index("id")
    qcols = [c for c in df_responses_subselected if "Q" in c]
    responses_matrix = df_responses_subselected[qcols].T.corr()
    votes_matrix = ParliamentAPI.generate_adjacency_matrix(df_votes_subselected, "jaccards", ["POOLT", "VASTU", "ERAPOOLETU"])
    votes_edges = votes_matrix.unstack().rename("corr_vote")
    responses_edges = responses_matrix.unstack().rename("corr_response")
    final_corr = pd.concat([votes_edges, responses_edges], axis=1).corr().loc["corr_vote", "corr_response"]
    #print(f"{dataset=}, {final_corr=}")
    dataset_to_score[dataset] = final_corr
pd.Series(dataset_to_score).rename("corr_Hääleandja").rename(index={"50": "50_EKRE_I_KE", "51": "51_RE_KE", "52": "52_RE_SDE_I", "XIV": "XIV Riigikogu (2019-2023)"})

50_EKRE_I_KE                 0.570644
51_RE_KE                     0.274521
52_RE_SDE_I                  0.291399
XIV Riigikogu (2019-2023)    0.467728
Name: corr_Hääleandja, dtype: float64